In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from depsurf import BuildVersion, FunctionGroups
import pickle


v = BuildVersion.filter()[-1]

with open(v.dwarf_path, "rb") as f:
    data = pickle.load(f)


func_groups = FunctionGroups(data)

In [24]:
from depsurf import OUTPUT_PATH, FunctionGroups

results = {}

for v in BuildVersion.filter():
    print(f"Processing {v.name}")
    all_groups = FunctionGroups.from_dump(v.dwarf_path)

    dup_groups = all_groups.filter(lambda name, loc_dict: len(loc_dict) > 1)
    dup_groups.dump_txt(OUTPUT_PATH / "dup" / v.name / "dup.txt")

    dup_header = {}
    dup_static = {}
    dup_mixed = {}

    for name, loc_dict in dup_groups.data.items():
        decl_locs = set(decl_loc for decl_loc, _ in loc_dict.keys())

        # Those are the functions that are declared in single header
        # but defined in multiple files
        if len(decl_locs) == 1:
            dup_header[name] = loc_dict
            continue

        if None in decl_locs:
            # Global functions have name collision with static functions
            dup_mixed[name] = loc_dict
        else:
            # Static functions have name collision with other static functions
            dup_static[name] = loc_dict

    dup_header = FunctionGroups(dup_header)
    dup_header.dump_txt(OUTPUT_PATH / "dup" / v.name / "dup_header.txt")
    dup_static = FunctionGroups(dup_static)
    dup_static.dump_txt(OUTPUT_PATH / "dup" / v.name / "dup_static.txt")
    dup_mixed = FunctionGroups(dup_mixed)
    dup_mixed.dump_txt(OUTPUT_PATH / "dup" / v.name / "dup_mixed.txt")

    num_no_dup = (
        all_groups.num_funcs
        - dup_header.num_funcs
        - dup_static.num_funcs
        - dup_mixed.num_funcs
    )

    results[v] = {
        "no_dup": num_no_dup,
        "dup_header": dup_header.num_funcs,
        "dup_static": dup_static.num_funcs,
        "dup_mixed": dup_mixed.num_funcs,
    }

Processing 4.4.0-21-generic-amd64
Processing 4.8.0-22-generic-amd64
Processing 4.10.0-19-generic-amd64
Processing 4.13.0-16-generic-amd64
Processing 4.15.0-20-generic-amd64
Processing 4.18.0-10-generic-amd64
Processing 5.0.0-13-generic-amd64
Processing 5.3.0-18-generic-amd64
Processing 5.4.0-26-generic-amd64
Processing 5.8.0-25-generic-amd64
Processing 5.11.0-16-generic-amd64
Processing 5.13.0-19-generic-amd64
Processing 5.15.0-25-generic-amd64
Processing 5.19.0-21-generic-amd64
Processing 6.2.0-20-generic-amd64
Processing 6.5.0-9-generic-amd64
Processing 6.8.0-22-generic-amd64


In [ ]:
import pandas as pd

df = pd.DataFrame(results).T

# plot the number of functions with name collision
import matplotlib.pyplot as plt

# df.plot(kind="bar", stacked=True)

# df